# SQL in Python - Connecting to and retrieving data from PostgreSQL

Previously, you have learned how to connect to a SQL database by using a SQL client such as DBeaver. Apart from connecting to databases, DBeaver also allows you to run SQL queries against the database, create new tables and populate them with data as well as retrieving the data.

Python also allows executing SQL queries and getting the result into a Python object, for example a Pandas data frame. Instead of exporting a .csv file from DBeaver you can directly get the data you need into Python and continue your work. In addition we can reduce the steps by connecting to the database from Python directly, eliminating the need for a separate SQL client.

After you have the data in Python in the required shape you can export the data into a .csv file. This file is for your own reference, please avoid sending .csv files around - database is the point of reference when it comes to data. 

Having a copy of a .csv file (or another format) can speed up your analysis work. Imagine that the query takes 25 minutes to run. If you made some mistakes in your Python code you might need to go back to the original dataset. Instead of having to rerun the SQL query and having to wait you can read in the .csv file you have previously saved on your hard disk into Python and continue with your analysis work. 

**In this notebook you will see 2 ways to connect to SQL-Databases and export the data to a CSV file**


## Creating a connection to a PostgreSQL database with Python

There are 2 python packages that are the "go-to" when it comes to connecting to SQL-Databases: `psycopg2` and `sqlalchemy` 

### Connecting via psycopg2

In [1]:
import pandas as pd
import psycopg2


In order to create a connection to our PostgreSQL database we need the following information:

- host = the address of the machine the database is hosted on
- port = the virtual gate number through which communication will be allowed
- database = the name of the database
- user = the name of the user
- password = the password of the user

Because we don't want that the database information is published on github we put it into a `.env` file which is added into the `.gitignore`. 
In these kind of files you can store information that is not supposed to be published.
With the `dotenv` package you can read the `.env` files and get the variables.
(We will share the file with you on Slack!)


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

DATABASE = os.getenv('DATABASE')
USER_DB = os.getenv('USER_DB')
PASSWORD = os.getenv('PASSWORD')
HOST = os.getenv('HOST')
PORT = os.getenv('PORT')

The function from the psycopg2 package to create a connection is called `connect()`.
`connect()` expects the parameters listed above as input in order to connect to the database.

In [3]:
# Create connection object conn
conn = psycopg2.connect(
    database=DATABASE,
    user=USER_DB,
    password=PASSWORD,
    host=HOST,
    port=PORT
)

### Retrieving data from the database with psycopg2

Before we can use our connection to get data, we have to create a cursor. A cursor allows Python code to execute PostgreSQL commmands in a database session.
A cursor has to be created with the `cursor()` method of our connection object conn.

In [4]:
cur = conn.cursor()

Now we can run SQL-Queries with `cur.execute('QUERY')` and then run `cur.fetchall()` to get the data:

In [5]:
cur.execute('SELECT * FROM eda.king_county_house_sales LIMIT 10')
cur.fetchall()

[(datetime.date(2014, 10, 13), 221900.0, 7129300520, 1),
 (datetime.date(2014, 12, 9), 538000.0, 6414100192, 2),
 (datetime.date(2015, 2, 25), 180000.0, 5631500400, 3),
 (datetime.date(2014, 12, 9), 604000.0, 2487200875, 4),
 (datetime.date(2015, 2, 18), 510000.0, 1954400510, 5),
 (datetime.date(2014, 5, 12), 1230000.0, 7237550310, 6),
 (datetime.date(2014, 6, 27), 257500.0, 1321400060, 7),
 (datetime.date(2015, 1, 15), 291850.0, 2008000270, 8),
 (datetime.date(2015, 4, 15), 229500.0, 2414600126, 9),
 (datetime.date(2015, 3, 12), 323000.0, 3793500160, 10)]

With `conn.close()` you can close the connection again.

In [6]:
#close the connection
conn.close()

But we want to work with the data. The easiest way is to import the data into pandas dataframes. We can use `pd.read_sql_query` or `pd.read_sql_table` or for convenience `pd.read_sql`.

This function is a convenience wrapper around read_sql_table and read_sql_query (for backward compatibility). It will delegate to the specific function depending on the provided input. A SQL query will be routed to read_sql_query , while a database table name will be routed to read_sql_table . Note that the delegated function might have more specific notes about their functionality not listed here.

In [11]:
# Open connection again because we closed it
conn = psycopg2.connect(
    database=DATABASE,
    user=USER_DB,
    password=PASSWORD,
    host=HOST,
    port=PORT
)

In [12]:
# import the data into a pandas dataframe
query_string = "SELECT * FROM eda.king_county_house_sales LIMIT 10"
df_psycopg = pd.read_sql(query_string, conn)

C:\Users\andre\AppData\Local\Temp\ipykernel_28384\1176423828.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_psycopg = pd.read_sql(query_string, conn)


In [13]:
#close the connection
conn.close()

In [14]:
df_psycopg.head()

,date,price,house_id,id
0,2014-10-13,221900.0,7129300520,1
1,2014-12-09,538000.0,6414100192,2
2,2015-02-25,180000.0,5631500400,3
3,2014-12-09,604000.0,2487200875,4
4,2015-02-18,510000.0,1954400510,5


In [17]:
#export the data to a csv-file
df_psycopg.to_csv('data/eda.csv',index=False)

### Connecting and retrieving data via SQLAlchemy

`sqlalchemy` works similarly. Here you have to create an engine with the database sting (a link that includes every information we entered in the conn object)

In [15]:
from sqlalchemy import create_engine

#read the database string from the .env
load_dotenv()

DB_STRING = os.getenv('DB_STRING')

db = create_engine(DB_STRING)

And then you can import that engine with a query into a pandas dataframe.

In [16]:
#import the data to a pandas dataframe
query_string = "SELECT * FROM eda.king_county_house_sales"
df_sqlalchemy = pd.read_sql(query_string, db)

In [17]:
df_sqlalchemy.head()

,date,price,house_id,id
0,2014-10-13,221900.0,7129300520,1
1,2014-12-09,538000.0,6414100192,2
2,2015-02-25,180000.0,5631500400,3
3,2014-12-09,604000.0,2487200875,4
4,2015-02-18,510000.0,1954400510,5


Because we don't want to run the queries over and over again we can export the data into a .csv file in order to use it in other notebooks as well. 

In [18]:
#export the data to a csv-file
df_sqlalchemy.to_csv('eda.csv',index=False)

Now we import the data of the second table into another csv file


In [19]:
#import the data to a pandas dataframe
query_string = "SELECT * FROM eda.king_county_house_details"
df_sqlalchemy = pd.read_sql(query_string, db)

In [ ]:
# write data into eda2.csv file
df_sqlalchemy.to_csv('eda2.csv', index=False)

## Let's generate 2 dataframes from those csv's and merge the data into one csv.

read out those csv's


In [34]:
#import the sales data from a csv-file 
df_import_sales = pd.read_csv('data/eda.csv', index_col=3)

In [35]:
df_import_sales

,date,price,house_id
id,,,
1,2014-10-13,221900.0,7129300520
2,2014-12-09,538000.0,6414100192
3,2015-02-25,180000.0,5631500400
4,2014-12-09,604000.0,2487200875
5,2015-02-18,510000.0,1954400510
...,...,...,...
21593,2014-05-21,360000.0,263000018
21594,2015-02-23,400000.0,6600060120
21595,2014-06-23,402101.0,1523300141


In [36]:
# import the details data from another csv file
df_import_details = pd.read_csv('data/eda2.csv', index_col=0)

In [37]:
df_import_details

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,,,,,,,,,,,,,,,,,,
1000102,6.0,3.00,2400.0,9373.0,2.0,NaN,0.0,3,7,2400.0,0.0,1991,0.0,98002,47.3262,-122.214,2060.0,7316.0
100100050,3.0,1.00,1320.0,11090.0,1.0,0.0,0.0,3,7,1320.0,0.0,1955,0.0,98155,47.7748,-122.304,1320.0,8319.0
1001200035,3.0,1.00,1350.0,7973.0,1.5,NaN,0.0,3,7,1350.0,0.0,1954,0.0,98188,47.4323,-122.292,1310.0,7491.0
1001200050,4.0,1.50,1260.0,7248.0,1.5,NaN,0.0,5,7,1260.0,0.0,1955,NaN,98188,47.4330,-122.292,1300.0,7732.0
1003000175,3.0,1.00,980.0,7606.0,1.0,0.0,0.0,3,7,980.0,0.0,1954,0.0,98188,47.4356,-122.290,980.0,8125.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993002177,3.0,2.50,1380.0,1547.0,3.0,0.0,0.0,3,8,1380.0,0.0,2000,NaN,98103,47.6908,-122.341,1380.0,1465.0
993002225,3.0,2.25,1520.0,1245.0,3.0,NaN,0.0,3,8,1520.0,0.0,2004,0.0,98103,47.6907,-122.340,1520.0,1470.0
993002247,3.0,2.25,1550.0,1469.0,3.0,0.0,0.0,3,8,1550.0,0.0,2004,0.0,98103,47.6911,-122.341,1520.0,1465.0


.. and merge the two into one

In [38]:
# Joining the DataFrames
df_merged = pd.merge(
    left=df_import_sales,         # The left DataFrame
    right=df_import_details,        # The right DataFrame
    left_on='house_id',         # Spalte im linken DF (df_personen)
    right_on='id',  # Spalte im rechten DF (df_haeuser)
    how='inner'
)

print(df_merged)

             date     price    house_id  bedrooms  bathrooms  sqft_living  \
0      2014-10-13  221900.0  7129300520       3.0       1.00       1180.0   
1      2014-12-09  538000.0  6414100192       3.0       2.25       2570.0   
2      2015-02-25  180000.0  5631500400       2.0       1.00        770.0   
3      2014-12-09  604000.0  2487200875       4.0       3.00       1960.0   
4      2015-02-18  510000.0  1954400510       3.0       2.00       1680.0   
...           ...       ...         ...       ...        ...          ...   
21592  2014-05-21  360000.0   263000018       3.0       2.50       1530.0   
21593  2015-02-23  400000.0  6600060120       4.0       2.50       2310.0   
21594  2014-06-23  402101.0  1523300141       2.0       0.75       1020.0   
21595  2015-01-16  400000.0   291310100       3.0       2.50       1600.0   
21596  2014-10-15  325000.0  1523300157       2.0       0.75       1020.0   

       sqft_lot  floors  waterfront  view  ...  grade  sqft_above  \
0     

In [41]:
df_merged.columns

Index(['date', 'price', 'house_id', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

now lets write this into our final csv file... 

In [42]:
df_merged.to_csv('data/King_County_House_prices_dataset.csv', index=False)